In [56]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [57]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [58]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv', index_col=[0])

In [59]:
dataset.head(10)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0.0,0.0,NaN,1.488361

In [60]:
dataset.user.unique().tolist() #all users

['U12',
 'U1',
 'U8',
 'U7',
 'U10',
 'U13',
 'U6',
 'U11',
 'U3',
 'U2',
 'U4',
 'U9',
 'U5']

In [61]:
dataset.columns

Index(['id', 'time', 'activityrecognition#0', 'activityrecognition#1',
       'android.sensor.accelerometer#mean', 'android.sensor.accelerometer#min',
       'android.sensor.accelerometer#max', 'android.sensor.accelerometer#std',
       'android.sensor.game_rotation_vector#mean',
       'android.sensor.game_rotation_vector#min',
       'android.sensor.game_rotation_vector#max',
       'android.sensor.game_rotation_vector#std',
       'android.sensor.gravity#mean', 'android.sensor.gravity#min',
       'android.sensor.gravity#max', 'android.sensor.gravity#std',
       'android.sensor.gyroscope#mean', 'android.sensor.gyroscope#min',
       'android.sensor.gyroscope#max', 'android.sensor.gyroscope#std',
       'android.sensor.gyroscope_uncalibrated#mean',
       'android.sensor.gyroscope_uncalibrated#min',
       'android.sensor.gyroscope_uncalibrated#max',
       'android.sensor.gyroscope_uncalibrated#std',
       'android.sensor.light#mean', 'android.sensor.light#min',
       'android.se

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [62]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5893 entries, 0 to 5892
Data columns (total 70 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   id                                               5893 non-null   int64  
 1   time                                             5893 non-null   float64
 2   activityrecognition#0                            0 non-null      float64
 3   activityrecognition#1                            5893 non-null   float64
 4   android.sensor.accelerometer#mean                5893 non-null   float64
 5   android.sensor.accelerometer#min                 5893 non-null   float64
 6   android.sensor.accelerometer#max                 5893 non-null   float64
 7   android.sensor.accelerometer#std                 5893 non-null   float64
 8   android.sensor.game_rotation_vector#mean         5094 non-null   float64
 9   android.sensor.game_rotation_v

In [63]:
dataset.describe()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.146752,14.137338,0.081745,15.141369,15.34

In [64]:
dataset.isnull().sum()

id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                       720
android.sensor.gyroscope#min                    

In [65]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-65-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000

In [66]:
dataset = dataset[(['accelerometer_mean',
                    'accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_min',
                    'gyroscope_max',
                    'gyroscope_std',
                    'proximity_std',
                    'sound_mean',
                    'user',
                    'target'
                 ])]

In [67]:
dataset.head()

,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,proximity_std,sound_mean,user,target
0,9.811476,9.758895,9.849411,0.014626,0.001651,0.000000,0.003533,0.000737,NaN,NaN,U12,Still
1,9.939207,7.707437,17.146631,1.775944,0.036326,0.011669,0.059388,0.020290,NaN,89.200210,U12,Car
2,9.827178,9.804817,9.849262,0.011199,0.001525,0.000000,0.002609,0.000713,NaN,NaN,U12,Still
3,9.673039,7.659674,12.304298,0.862553,0.036440,0.020463,0.052512,0.010553,NaN,87.470377,U12,Car
4,9.993466,8.965621,10.891645,0.504117,0.183202,0.020667,0.380875,0.098819,NaN,89.770732,U1,Car


In [68]:
# split user = u12 & u3 as test/train
dataset_test = dataset[(dataset['user'] == 'U2')|(dataset['user'] == 'U3')|(dataset['user'] == 'U4')]
dataset_train = dataset[(dataset['user'] != 'U2')&(dataset['user'] != 'U3')&(dataset['user'] != 'U4')]

In [69]:
dataset_train.sample(10)

,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,proximity_std,sound_mean,user,target
167,9.884823,9.766190,10.027158,0.078935,0.005720,0.001145,0.012858,0.003141,NaN,86.298109,U1,Car
4283,9.490994,7.559635,11.552176,1.079596,0.183706,0.042177,0.345499,0.070508,NaN,89.738806,U1,Bus
1538,9.359706,6.766191,11.282024,1.091400,0.198715,0.052771,0.670915,0.154882,NaN,84.248014,U13,Car
3302,9.816696,9.777663,9.854402,0.016447,0.001591,0.000000,0.003196,0.000803,NaN,NaN,U12,Still
1871,9.837159,9.265629,11.499277,0.332854,0.280135,0.170610,0.554274,0.183529,NaN,59.854013,U10,Still
4339,10.214287,9.868533,10.467905,0.231013,0.024517,0.008875,0.056552,0.014016,NaN,89.740787,U1,Train
4358,9.993635,7.008135,12.480351,1.242973,0.047007,0.010706,0.099253,0.024945,NaN,83.872508,U12,Car
5529,10.065256,8.057055,11.597801,0.443210,NaN,NaN,NaN,NaN,NaN,62.934960,U6,Bus
1353,9.987935,9.975651,10.008633,0.009564,0.005833,0.002110,0.009912,0.002262,NaN,53.121964,U1,Still
1127,9.803954,9.774109,9.844461,0.017746,0.001729,0.000000,0.003369,0.000878,NaN,NaN,U12,Still


In [79]:
dataset_train = dataset_train.fillna(0)

In [80]:
dataset_train.head()

,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,proximity_std,sound_mean,target
0,9.811476,9.758895,9.849411,0.014626,0.001651,0.000000,0.003533,0.000737,0.0,0.000000,Still
1,9.939207,7.707437,17.146631,1.775944,0.036326,0.011669,0.059388,0.020290,0.0,89.200210,Car
2,9.827178,9.804817,9.849262,0.011199,0.001525,0.000000,0.002609,0.000713,0.0,0.000000,Still
3,9.673039,7.659674,12.304298,0.862553,0.036440,0.020463,0.052512,0.010553,0.0,87.470377,Car
4,9.993466,8.965621,10.891645,0.504117,0.183202,0.020667,0.380875,0.098819,0.0,89.770732,Car


In [81]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

KeyError: "['user'] not found in axis"

In [82]:
num_vars = ['accelerometer_mean',
                    'accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_min',
                    'gyroscope_max',
                    'gyroscope_std',
                    'proximity_std',
                    'sound_mean',]

In [74]:
# YOUR CODE HERE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import QuantileTransformer, StandardScaler
num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputing', KNNImputer(n_neighbors=3)),
    ('scaler', StandardScaler())
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputing',
                                                  KNNImputer(n_neighbors=3)),
                                                 ('scaler', StandardScaler())]),
                                 ['accelerometer_mean', 'accelerometer_min',
                                  'accelerometer_max', 'accelerometer_std',
                                  'gyroscope_mean', 'gyroscope_min',
                                  'gyroscope_max', 'gyroscope_std',
                                  'proximity_std', 'sound_mean'])])

In [75]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [76]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputing',
                                                                   KNNImputer(n_neighbors=3)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['accelerometer_mean',
                                                   'accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_min',
                                                   'gyroscope_max',
                                                   'gyroscope_std',
                                                   'proximity_std',
                                                   'sound_mean'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [77]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_val = dataset_test.drop(['target'], axis=1)
y_val = dataset_test.target

In [78]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:40:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.085951
0:	learn: 1.4922299	total: 41.1ms	remaining: 41s
1:	learn: 1.3940858	total: 83.2ms	remaining: 41.5s
2:	learn: 1.3197341	total: 122ms	remaining: 40.6s
3:	learn: 1.2528703	total: 297ms	remaining: 1m 14s
4:	learn: 1.1905822	total: 375ms	remaining: 1m 14s
5:	learn: 1.1411741	total: 440ms	remaining: 1m 12s
6:	learn: 1.0998105	total: 491ms	remaining: 1m 9s
7:	learn: 1.0638273	total: 580ms	remaining: 1m 11s
8:	learn: 1.0315357	total: 651ms	remaining: 1m 11s
9:	learn: 1.0021496	total: 700ms	remaining: 1m 9s
10:	learn: 0.9728695	total: 765ms	remaining: 1m 8s
11:	learn: 0.9487751	total: 832ms	remaining: 1m 8s
12:	learn: 0.9273320	total: 927ms	remaining: 1m 10s
13:	learn: 0.9094629	total: 968ms	remaining: 1m 8s
14:	lea

153:	learn: 0.4717521	total: 12.3s	remaining: 1m 7s
154:	learn: 0.4705679	total: 12.5s	remaining: 1m 8s
155:	learn: 0.4693334	total: 12.6s	remaining: 1m 7s
156:	learn: 0.4689146	total: 12.7s	remaining: 1m 7s
157:	learn: 0.4683369	total: 12.7s	remaining: 1m 7s
158:	learn: 0.4669155	total: 12.8s	remaining: 1m 7s
159:	learn: 0.4651565	total: 12.8s	remaining: 1m 7s
160:	learn: 0.4643385	total: 12.9s	remaining: 1m 7s
161:	learn: 0.4627229	total: 13s	remaining: 1m 7s
162:	learn: 0.4617133	total: 13.1s	remaining: 1m 7s
163:	learn: 0.4609671	total: 13.2s	remaining: 1m 7s
164:	learn: 0.4600430	total: 13.2s	remaining: 1m 6s
165:	learn: 0.4593842	total: 13.3s	remaining: 1m 7s
166:	learn: 0.4580822	total: 13.4s	remaining: 1m 6s
167:	learn: 0.4574401	total: 13.4s	remaining: 1m 6s
168:	learn: 0.4565922	total: 13.5s	remaining: 1m 6s
169:	learn: 0.4558354	total: 13.6s	remaining: 1m 6s
170:	learn: 0.4551490	total: 13.6s	remaining: 1m 5s
171:	learn: 0.4540174	total: 13.7s	remaining: 1m 5s
172:	learn: 0.

316:	learn: 0.3600996	total: 22.4s	remaining: 48.2s
317:	learn: 0.3598913	total: 22.4s	remaining: 48.1s
318:	learn: 0.3595378	total: 22.5s	remaining: 48s
319:	learn: 0.3590782	total: 22.5s	remaining: 47.9s
320:	learn: 0.3587880	total: 22.7s	remaining: 48s
321:	learn: 0.3584260	total: 22.7s	remaining: 47.8s
322:	learn: 0.3575312	total: 22.8s	remaining: 47.7s
323:	learn: 0.3571732	total: 22.8s	remaining: 47.6s
324:	learn: 0.3568266	total: 22.9s	remaining: 47.5s
325:	learn: 0.3564408	total: 22.9s	remaining: 47.4s
326:	learn: 0.3556898	total: 23s	remaining: 47.3s
327:	learn: 0.3552162	total: 23s	remaining: 47.2s
328:	learn: 0.3547084	total: 23.1s	remaining: 47s
329:	learn: 0.3541976	total: 23.1s	remaining: 47s
330:	learn: 0.3534483	total: 23.2s	remaining: 46.9s
331:	learn: 0.3527758	total: 23.3s	remaining: 46.8s
332:	learn: 0.3520106	total: 23.3s	remaining: 46.6s
333:	learn: 0.3513549	total: 23.3s	remaining: 46.5s
334:	learn: 0.3506978	total: 23.4s	remaining: 46.4s
335:	learn: 0.3497331	to

475:	learn: 0.2916280	total: 32.7s	remaining: 36s
476:	learn: 0.2913549	total: 32.8s	remaining: 36s
477:	learn: 0.2911635	total: 32.9s	remaining: 35.9s
478:	learn: 0.2907206	total: 32.9s	remaining: 35.8s
479:	learn: 0.2904496	total: 33s	remaining: 35.8s
480:	learn: 0.2900418	total: 33.1s	remaining: 35.7s
481:	learn: 0.2898019	total: 33.1s	remaining: 35.6s
482:	learn: 0.2897091	total: 33.2s	remaining: 35.5s
483:	learn: 0.2894600	total: 33.2s	remaining: 35.4s
484:	learn: 0.2891663	total: 33.3s	remaining: 35.3s
485:	learn: 0.2886698	total: 33.3s	remaining: 35.2s
486:	learn: 0.2882134	total: 33.4s	remaining: 35.1s
487:	learn: 0.2877018	total: 33.4s	remaining: 35.1s
488:	learn: 0.2874078	total: 33.5s	remaining: 35s
489:	learn: 0.2871749	total: 33.5s	remaining: 34.9s
490:	learn: 0.2869455	total: 33.6s	remaining: 34.8s
491:	learn: 0.2862169	total: 33.7s	remaining: 34.8s
492:	learn: 0.2858216	total: 33.7s	remaining: 34.7s
493:	learn: 0.2856032	total: 33.8s	remaining: 34.6s
494:	learn: 0.285335

636:	learn: 0.2453564	total: 42.1s	remaining: 24s
637:	learn: 0.2449309	total: 42.2s	remaining: 23.9s
638:	learn: 0.2445993	total: 42.2s	remaining: 23.9s
639:	learn: 0.2443342	total: 42.3s	remaining: 23.8s
640:	learn: 0.2439513	total: 42.4s	remaining: 23.7s
641:	learn: 0.2435464	total: 42.4s	remaining: 23.6s
642:	learn: 0.2432897	total: 42.5s	remaining: 23.6s
643:	learn: 0.2430256	total: 42.5s	remaining: 23.5s
644:	learn: 0.2429443	total: 42.6s	remaining: 23.5s
645:	learn: 0.2428001	total: 42.7s	remaining: 23.4s
646:	learn: 0.2422574	total: 42.8s	remaining: 23.3s
647:	learn: 0.2420783	total: 42.8s	remaining: 23.3s
648:	learn: 0.2419328	total: 43s	remaining: 23.3s
649:	learn: 0.2416931	total: 43.1s	remaining: 23.2s
650:	learn: 0.2414893	total: 43.1s	remaining: 23.1s
651:	learn: 0.2412563	total: 43.2s	remaining: 23.1s
652:	learn: 0.2409801	total: 43.3s	remaining: 23s
653:	learn: 0.2406993	total: 43.4s	remaining: 23s
654:	learn: 0.2404078	total: 43.5s	remaining: 22.9s
655:	learn: 0.240214

796:	learn: 0.2079203	total: 52.2s	remaining: 13.3s
797:	learn: 0.2076610	total: 52.3s	remaining: 13.2s
798:	learn: 0.2073904	total: 52.3s	remaining: 13.2s
799:	learn: 0.2070241	total: 52.4s	remaining: 13.1s
800:	learn: 0.2067644	total: 52.4s	remaining: 13s
801:	learn: 0.2066403	total: 52.5s	remaining: 13s
802:	learn: 0.2064232	total: 52.6s	remaining: 12.9s
803:	learn: 0.2061427	total: 52.6s	remaining: 12.8s
804:	learn: 0.2059804	total: 52.7s	remaining: 12.8s
805:	learn: 0.2057304	total: 52.7s	remaining: 12.7s
806:	learn: 0.2055024	total: 52.8s	remaining: 12.6s
807:	learn: 0.2053247	total: 52.8s	remaining: 12.5s
808:	learn: 0.2052051	total: 52.8s	remaining: 12.5s
809:	learn: 0.2049644	total: 52.9s	remaining: 12.4s
810:	learn: 0.2047373	total: 52.9s	remaining: 12.3s
811:	learn: 0.2045828	total: 53s	remaining: 12.3s
812:	learn: 0.2043749	total: 53s	remaining: 12.2s
813:	learn: 0.2041509	total: 53.1s	remaining: 12.1s
814:	learn: 0.2038888	total: 53.1s	remaining: 12.1s
815:	learn: 0.203761

956:	learn: 0.1782425	total: 1m 1s	remaining: 2.75s
957:	learn: 0.1780462	total: 1m 1s	remaining: 2.69s
958:	learn: 0.1779494	total: 1m 1s	remaining: 2.63s
959:	learn: 0.1778310	total: 1m 1s	remaining: 2.56s
960:	learn: 0.1775341	total: 1m 1s	remaining: 2.5s
961:	learn: 0.1774241	total: 1m 1s	remaining: 2.43s
962:	learn: 0.1771909	total: 1m 1s	remaining: 2.37s
963:	learn: 0.1769812	total: 1m 1s	remaining: 2.31s
964:	learn: 0.1767971	total: 1m 1s	remaining: 2.24s
965:	learn: 0.1766816	total: 1m 1s	remaining: 2.18s
966:	learn: 0.1766169	total: 1m 1s	remaining: 2.11s
967:	learn: 0.1764909	total: 1m 1s	remaining: 2.05s
968:	learn: 0.1764010	total: 1m 2s	remaining: 1.98s
969:	learn: 0.1762068	total: 1m 2s	remaining: 1.92s
970:	learn: 0.1760652	total: 1m 2s	remaining: 1.85s
971:	learn: 0.1759568	total: 1m 2s	remaining: 1.79s
972:	learn: 0.1757773	total: 1m 2s	remaining: 1.73s
973:	learn: 0.1756550	total: 1m 2s	remaining: 1.66s
974:	learn: 0.1754326	total: 1m 2s	remaining: 1.6s
975:	learn: 0.

,Model,Accuracy,Bal Acc.,Time
1,Extra Trees,50.060314,41.574365,13.929369
2,Skl GBM,49.336550,44.899574,46.351051
3,Skl HistGBM,49.336550,44.899574,44.250966
4,LightGBM,48.009650,40.734200,27.826177
5,XGBoost,47.889023,41.764278,46.115213
6,CatBoost,45.235223,41.625459,79.979524
7,Random Forest,44.752714,44.833522,13.756057
8,AdaBoost,38.962606,40.146356,9.494630
9,Decision Tree,31.604343,37.380187,12.123288
